1. 确定基本的变量（即data sheet）

In [1165]:
# scheme O
road_length__O = 5 # km
average_speed__O = 22 # km/h
AADT__O = 15000 # vehicles/day

# scheme A
road_length__A = 9 # km
average_speed__A = 65 # km/h
average_speed__A_O = 28 # km/h
AADT__A = 13000 # vehicles/day
AADT__A_O = 4000 # vehicles/day
construction_cost__A = 90000000 # pounds
maintainance_cost_A = 10000 # pounds/year

# scheme B
road_length_B = 11 # km
average_speed__B = 65 # km/h
average_speed_B_O = 28 # km/h
AADT__B = 13500 # vehicles/day
AADT__B_O = 3500 # vehicles/day
construction_cost__B = 77000000 # pounds
maintenance_cost__B = 10000 # pounds/year

# low growth scenario
growth_rate__low = 0.01 # per year

# high growth scenario
growth_rate__high = 0.02 # per year


计算不同年份的AADT，存储成一个字典

In [1166]:
# 计算不同年份的AADT，存储成一个字典
# 生成2031年到2090年的AADT
AADT__O_dict_low = {}
AADT__O_dict_high = {}
AADT__A_dict_low = {}
AADT__A_dict_high = {}
AADT__A_O_dict_low = {}
AADT__A_O_dict_high = {}
AADT__B_dict_low = {}
AADT__B_dict_high = {}
AADT__B_O_dict_low = {}
AADT__B_O_dict_high = {}
# 前二十年增长，后面保持不变
for year in range(2031, 2091):
    if year - 2030 <= 20:
        AADT__O_dict_low[year] = AADT__O * (1 + growth_rate__low) ** (year - 2030)
        AADT__O_dict_high[year] = AADT__O * (1 + growth_rate__high) ** (year - 2030)
        AADT__A_dict_low[year] = AADT__A * (1 + growth_rate__low) ** (year - 2031)
        AADT__A_dict_high[year] = AADT__A * (1 + growth_rate__high) ** (year - 2031)
        AADT__A_O_dict_low[year] = AADT__A_O * (1 + growth_rate__low) ** (year - 2031)
        AADT__A_O_dict_high[year] = AADT__A_O * (1 + growth_rate__high) ** (year - 2031)
        AADT__B_dict_low[year] = AADT__B * (1 + growth_rate__low) ** (year - 2031)
        AADT__B_dict_high[year] = AADT__B * (1 + growth_rate__high) ** (year - 2031)
        AADT__B_O_dict_low[year] = AADT__B_O * (1 + growth_rate__low) ** (year - 2031)
        AADT__B_O_dict_high[year] = AADT__B_O * (1 + growth_rate__high) ** (year - 2031)
    else:
        AADT__O_dict_low[year] = AADT__O * (1 + growth_rate__low) ** 20
        AADT__O_dict_high[year] = AADT__O * (1 + growth_rate__high) ** 20
        AADT__A_dict_low[year] = AADT__A * (1 + growth_rate__low) ** 19
        AADT__A_dict_high[year] = AADT__A * (1 + growth_rate__high) ** 19
        AADT__A_O_dict_low[year] = AADT__A_O * (1 + growth_rate__low) ** 19
        AADT__A_O_dict_high[year] = AADT__A_O * (1 + growth_rate__high) ** 19
        AADT__B_dict_low[year] = AADT__B * (1 + growth_rate__low) ** 19
        AADT__B_dict_high[year] = AADT__B * (1 + growth_rate__high) ** 19
        AADT__B_O_dict_low[year] = AADT__B_O * (1 + growth_rate__low) ** 19
        AADT__B_O_dict_high[year] = AADT__B_O * (1 + growth_rate__high) ** 19

        

计算GC中旅行时间节省的部分；
这里工作和非工作的数值是一致的，故不区分

In [1167]:
the_all_week_value_of_time_for_an_average_car = 10.79 # pounds/h
the_coefficient_of_converting_time_to_2030 = 1.5893 # using the GDP deflator
the_value_of_time_2030 = the_all_week_value_of_time_for_an_average_car * the_coefficient_of_converting_time_to_2030
Income_growth_previous_efficiency = 125.57 / 104.88 # using the annual parameters in WebTAG
the_value_of_time_2030 = the_value_of_time_2030 * Income_growth_previous_efficiency
long_term_growth_rate = 0.015
the_value_of_time_dict = {}
for year in range(2031, 2091):
    the_value_of_time_dict[year] = the_value_of_time_2030 * (1 + long_term_growth_rate) ** (year - 2030)    

# 定义关于旅行时间的感知成本
GC_for_journey_time_O = {}
GC_for_journey_time_A = {}
GC_for_journey_time_A_O = {}
GC_for_journey_time_B = {}
GC_for_journey_time_B_O = {}

for year in range(2031, 2091):
    GC_for_journey_time_O[year] = the_value_of_time_dict[year] * road_length__O / average_speed__O
    GC_for_journey_time_A[year] = the_value_of_time_dict[year] * road_length__A / average_speed__A
    GC_for_journey_time_A_O[year] = the_value_of_time_dict[year] * road_length__O / average_speed__A_O
    GC_for_journey_time_B[year] = the_value_of_time_dict[year] * road_length_B / average_speed__B
    GC_for_journey_time_B_O[year] = the_value_of_time_dict[year] * road_length__O / average_speed_B_O



    


计算GC中的燃料成本和非燃料成本，注意非工作非燃料成本不用半数法(先不考虑)，而非工作燃料成本其实等于工作燃料成本

In [1168]:
import csv

# 初始化字典
fuel_price_per_kwh = {}
fuel_effieciency = {}
emission_factor = {}
money_value_of_emission = {}
petrol_fuel_price_65 = {}
petrol_fuel_price_28 = {}
petrol_fuel_price_22 = {}
non_work_petrol_fuel_price_65 = {} 
non_work_petrol_fuel_price_28 = {}
non_work_petrol_fuel_price_22 = {}

with open('voc.csv', mode='r', encoding='utf-8') as file:
    reader = csv.reader(file)
    next(reader)  # 跳过标题行
    for rows in reader:
        year = int(rows[0])
        fuel_price_per_kwh[year] = float(rows[1])
        fuel_effieciency[year] = float(rows[2])
        emission_factor[year] = float(rows[3])
        money_value_of_emission[year] = float(rows[4])
        petrol_fuel_price_65[year] = float(rows[5])
        petrol_fuel_price_28[year] = float(rows[6])
        petrol_fuel_price_22[year] = float(rows[7])
        non_work_petrol_fuel_price_65[year] = float(rows[8])
        non_work_petrol_fuel_price_28[year] = float(rows[9])
        non_work_petrol_fuel_price_22[year] = float(rows[10])

GC_for_fuel_cost_O = {} 
GC_for_fuel_cost_A = {}
GC_for_fuel_cost_B = {}

for year in range(2031, 2091):
    GC_for_fuel_cost_O[year] = fuel_price_per_kwh[year] * fuel_effieciency[year] * road_length__O * 1.5893
    GC_for_fuel_cost_A[year] = fuel_price_per_kwh[year] * fuel_effieciency[year] * road_length__A * 1.5893
    GC_for_fuel_cost_B[year] = fuel_price_per_kwh[year] * fuel_effieciency[year] * road_length_B * 1.5893

def get_non_fuel_efficiency(type, velocity):
    if type == '1':
        return 1.157 + 135.946 / velocity
    elif type == '0':
        return 1.157 

GC_for_non_fuel_non_Work_O = get_non_fuel_efficiency('0', average_speed__O) * road_length__O * 1.5893
GC_for_non_fuel_non_Work_A = get_non_fuel_efficiency('0', average_speed__A) * road_length__A * 1.5893
GC_for_non_fuel_non_Work_A_O = get_non_fuel_efficiency('0', average_speed__A_O) * road_length__O * 1.5893
GC_for_non_fuel_non_Work_B = get_non_fuel_efficiency('0', average_speed__B) * road_length_B * 1.5893
GC_for_non_fuel_non_Work_B_O = get_non_fuel_efficiency('0', average_speed_B_O) * road_length__O * 1.5893
GC_for_non_fuel_work_O = get_non_fuel_efficiency('1', average_speed__O) * road_length__O* 1.5893
GC_for_non_fuel_work_A = get_non_fuel_efficiency('1', average_speed__A) * road_length__A * 1.5893
GC_for_non_fuel_work_A_O = get_non_fuel_efficiency('1', average_speed__A_O) * road_length__O * 1.5893
GC_for_non_fuel_work_B = get_non_fuel_efficiency('1', average_speed__B) * road_length_B * 1.5893
GC_for_non_fuel_work_B_O = get_non_fuel_efficiency('1', average_speed_B_O) * road_length__O * 1.5893

GC_O_work = {}
GC_A_work = {}
GC_A_O_work = {}
GC_B_work = {}
GC_B_O_work = {}

for year in range(2031, 2091):
    GC_O_work[year] = GC_for_journey_time_O[year] + GC_for_fuel_cost_O[year] / 100 + GC_for_non_fuel_work_O /100
    GC_A_work[year] = GC_for_journey_time_A[year] + GC_for_fuel_cost_A[year] / 100 + GC_for_non_fuel_work_A /100
    GC_A_O_work[year] = GC_for_journey_time_A_O[year] + GC_for_fuel_cost_A[year] / 100 + GC_for_non_fuel_work_A_O /100
    GC_B_work[year] = GC_for_journey_time_B[year] + GC_for_fuel_cost_B[year] / 100 + GC_for_non_fuel_work_B /100
    GC_B_O_work[year] = GC_for_journey_time_B_O[year] + GC_for_fuel_cost_B[year] / 100 + GC_for_non_fuel_work_B_O /100

print(list(GC_for_journey_time_O.values())[:5])
print(list(GC_for_journey_time_A.values())[:5])
print(list(GC_for_journey_time_A_O.values())[:5])
print(list(GC_for_journey_time_B.values())[:5])
print(list(GC_for_journey_time_B_O.values())[:5])

    




[4.736242269285785, 4.80728590332507, 4.879395191874946, 4.952586119753071, 5.026874911549365]
[2.8854645209802623, 2.928746488794966, 2.97267768612689, 3.0172678514187936, 3.062526869190075]
[3.7213332115816877, 3.7771532097554124, 3.833810507901743, 3.8913176655202695, 3.9496874305030727]
[3.526678858975876, 3.579579041860514, 3.6332727274884213, 3.6877718184007477, 3.7430883956767587]
[3.7213332115816877, 3.7771532097554124, 3.833810507901743, 3.8913176655202695, 3.9496874305030727]


先只计算工作部分, 计算结果以英镑为单位

In [1169]:
# 根据ROH的公式计算user benefit
def get_ROH(GC_new, GC_new_old, GC_old, AADT_new, AADT_new_old, AADT_old):
    '''
    GC_new: GC_A(B)
    GC_new_old: GC_A(B)_O
    GC_old: GC_O
    AADT_new: AADT_A(B)
    AADT_new_old: AADT_A(B)_O
    AADT_old: AADT_O
    '''
    GC_average = (GC_new * AADT_new + GC_new_old * AADT_new_old) / (AADT_new + AADT_new_old)
    return (GC_old - GC_average) * (AADT_new + AADT_new_old + AADT_old) / 2 * 365.25

user_benefit_A_work_low = {}
user_benefit_A_work_high = {}
user_benefit_B_work_low = {}
user_benefit_B_work_high = {}


for year in range(2031, 2091):
    user_benefit_A_work_low[year] = get_ROH(GC_A_work[year], GC_A_O_work[year], GC_O_work[year], 0.053 * AADT__A_dict_low[year], 0.053 * AADT__A_O_dict_low[year], 0.053 * AADT__O_dict_low[year])
    user_benefit_A_work_high[year] = get_ROH(GC_A_work[year], GC_A_O_work[year], GC_O_work[year], 0.053 * AADT__A_dict_high[year], 0.053 * AADT__A_O_dict_high[year], 0.053 * AADT__O_dict_high[year])
    user_benefit_B_work_low[year] = get_ROH(GC_B_work[year], GC_B_O_work[year], GC_O_work[year], 0.053 * AADT__B_dict_low[year], 0.053 * AADT__B_O_dict_low[year], 0.053 * AADT__O_dict_low[year])
    user_benefit_B_work_high[year] = get_ROH(GC_B_work[year], GC_B_O_work[year], GC_O_work[year], 0.053 * AADT__B_dict_high[year], 0.053 * AADT__B_O_dict_high[year], 0.053 * AADT__O_dict_high[year])



下面这部分考虑的是将未来收益折现到2030年

In [1170]:
def discount_to_2030(values, end_year):
    """
    对给定的每年收益按年折现到2030年。
    对于距2030年30年内的年份使用3.5%的折现率，超过30年的部分使用3%的折现率。
    
    参数:
    values: 一个字典，键为年份，值为那一年的收益。
    
    返回:
    2030年的折现总值。
    """
    discount_value = 0  # 初始化折现后的总值
    for year, value in values.items():
        if year > end_year:
            continue

        years_diff = year - 2030  # 计算与2030年的年差
        if years_diff <= 30:
            # 对于30年及以内的部分，使用3.5%的折现率
            discount_factor = (1 + 0.035) ** years_diff
        else:
            # 对于超过30年的部分，前30年使用3.5%，之后使用3%
            discount_factor_30 = (1 + 0.035) ** 30
            discount_factor_after_30 = (1 + 0.03) ** (years_diff - 30)
            discount_factor = discount_factor_30 * discount_factor_after_30
        
        discount_value += value / discount_factor  # 将该年的收益折现到2030年的价值，并累加到总值中
    
    return discount_value

print(discount_to_2030(user_benefit_A_work_high, 2060))

14224945.19950175


下面考虑将非工作燃料成本的ROH

In [1171]:
GC_O_non_work_fuel = {}
GC_A_non_work_fuel = {}
GC_A_O_non_work_fuel = {}
GC_B_non_work_fuel = {}
GC_B_O_non_work_fuel = {}

for year in range(2031, 2091):
    GC_O_non_work_fuel[year] = GC_for_journey_time_O[year] + GC_for_fuel_cost_O[year] / 100
    GC_A_non_work_fuel[year] = GC_for_journey_time_A[year] + GC_for_fuel_cost_A[year] / 100
    GC_A_O_non_work_fuel[year] = GC_for_journey_time_A_O[year] + GC_for_fuel_cost_A[year] / 100
    GC_B_non_work_fuel[year] = GC_for_journey_time_B[year] + GC_for_fuel_cost_B[year] / 100
    GC_B_O_non_work_fuel[year] = GC_for_journey_time_B_O[year] + GC_for_fuel_cost_B[year] / 100

user_benefit_A_non_work_low = {}
user_benefit_A_non_work_high = {}
user_benefit_B_non_work_low = {}
user_benefit_B_non_work_high = {}

for year in range(2031, 2091):
    user_benefit_A_non_work_low[year] = get_ROH(GC_A_non_work_fuel[year], GC_A_O_non_work_fuel[year], GC_O_non_work_fuel[year], 0.947 * AADT__A_dict_low[year], 0.947 * AADT__A_O_dict_low[year], 0.947 * AADT__O_dict_low[year])
    user_benefit_A_non_work_high[year] = get_ROH(GC_A_non_work_fuel[year], GC_A_O_non_work_fuel[year], GC_O_non_work_fuel[year], 0.947 * AADT__A_dict_high[year], 0.947 * AADT__A_O_dict_high[year], 0.947 * AADT__O_dict_high[year])
    user_benefit_B_non_work_low[year] = get_ROH(GC_B_non_work_fuel[year], GC_B_O_non_work_fuel[year], GC_O_non_work_fuel[year], 0.947 * AADT__B_dict_low[year], 0.947 * AADT__B_O_dict_low[year], 0.947 * AADT__O_dict_low[year])
    user_benefit_B_non_work_high[year] = get_ROH(GC_B_non_work_fuel[year], GC_B_O_non_work_fuel[year], GC_O_non_work_fuel[year], 0.947 * AADT__B_dict_high[year], 0.947 * AADT__B_O_dict_high[year], 0.947 * AADT__O_dict_high[year])
    
    

下面计算非工作非燃料的净收益

In [1172]:
user_benefit_non_work_non_fuel_A_low = {}
user_benefit_non_work_non_fuel_A_high = {}
user_benefit_non_work_non_fuel_B_low = {}
user_benefit_non_work_non_fuel_B_high = {}

for year in range(2031, 2091):
    user_benefit_non_work_non_fuel_A_low[year] = (GC_for_non_fuel_non_Work_A / 100 * AADT__A_dict_low[year]  + GC_for_non_fuel_non_Work_A_O / 100 * AADT__A_O_dict_low[year] \
                                                - GC_for_non_fuel_non_Work_O / 100 * AADT__O_dict_low[year]) * 365.25 
    user_benefit_non_work_non_fuel_B_low[year] = (GC_for_non_fuel_non_Work_B / 100 * AADT__B_dict_low[year]  + GC_for_non_fuel_non_Work_B_O / 100 * AADT__B_O_dict_low[year] \
                                                - GC_for_non_fuel_non_Work_O / 100 * AADT__O_dict_low[year]) * 365.25
    user_benefit_non_work_non_fuel_A_high[year] = (GC_for_non_fuel_non_Work_A / 100 * AADT__A_dict_high[year]  + GC_for_non_fuel_non_Work_A_O / 100 * AADT__A_O_dict_high[year] \
                                                - GC_for_non_fuel_non_Work_O / 100 * AADT__O_dict_high[year]) * 365.25
    user_benefit_non_work_non_fuel_B_high[year] = (GC_for_non_fuel_non_Work_B / 100 * AADT__B_dict_high[year]  + GC_for_non_fuel_non_Work_B_O / 100 * AADT__B_O_dict_high[year] \
                                                - GC_for_non_fuel_non_Work_O / 100 * AADT__O_dict_high[year]) * 365.25



下面计算税收的损失，使用二分之一法则整合到前面计算的收益中（算作收益的损失而非成本）

In [1173]:
t = 0.19
tf1 = 0
tf = 0.05
tn1 = 0
tn = 0.19
# formula in A.5
def calculate_tax(type, ROH_fuel, ROH_non_fuel):
    # 0 for non-work, 1 for work
    if type == 1:
        return ROH_fuel * tf1 * (1 + t) / (1 + tf1) + ROH_non_fuel * tn1 * (1 + t) / (1 + tn1)
    elif type == 0:
        return ROH_fuel * tf * (tf -t) / (1 + tf) + ROH_non_fuel * tn * (tn - t) / (1 + tn)
    




In [1174]:
F_work_A_low = {}
F_work_A_high = {}
F_work_B_low = {}
F_work_B_high = {}
N_work_A_low ={}
N_work_A_high = {}
N_work_B_low = {}
N_work_B_high = {}
F_non_work_A_low = {}
F_non_work_A_high = {}
F_non_work_B_low = {}
F_non_work_B_high = {}
N_non_work_A_low = {}
N_non_work_A_high = {}
N_non_work_B_low = {}
N_non_work_B_high = {}
for year in range(2031, 2091):
    F_work_A_low[year] = get_ROH(GC_for_fuel_cost_A[year], GC_for_fuel_cost_O[year], GC_for_fuel_cost_O[year], 0.053 * AADT__A_dict_low[year], 0.053 * AADT__A_O_dict_low[year], 0.053 * AADT__O_dict_low[year])
    F_work_A_high[year] = get_ROH(GC_for_fuel_cost_A[year], GC_for_fuel_cost_O[year], GC_for_fuel_cost_O[year], 0.053 * AADT__A_dict_high[year], 0.053 * AADT__A_O_dict_high[year], 0.053 * AADT__O_dict_high[year])
    F_work_B_low[year] = get_ROH(GC_for_fuel_cost_B[year], GC_for_fuel_cost_O[year], GC_for_fuel_cost_O[year], 0.053 * AADT__B_dict_low[year], 0.053 * AADT__B_O_dict_low[year], 0.053 * AADT__O_dict_low[year])
    F_work_B_high[year] = get_ROH(GC_for_fuel_cost_B[year], GC_for_fuel_cost_O[year], GC_for_fuel_cost_O[year], 0.053 * AADT__B_dict_high[year], 0.053 * AADT__B_O_dict_high[year], 0.053 * AADT__O_dict_high[year])
    N_work_A_low[year] = get_ROH(GC_for_non_fuel_work_A, GC_for_non_fuel_work_A_O, GC_for_non_fuel_work_O, 0.053 * AADT__A_dict_low[year], 0.053 * AADT__A_O_dict_low[year], 0.053 * AADT__O_dict_low[year])
    N_work_A_high[year] = get_ROH(GC_for_non_fuel_work_A, GC_for_non_fuel_work_A_O, GC_for_non_fuel_work_O, 0.053 * AADT__A_dict_high[year], 0.053 * AADT__A_O_dict_high[year], 0.053 * AADT__O_dict_high[year])
    N_work_B_low[year] = get_ROH(GC_for_non_fuel_work_B, GC_for_non_fuel_work_B_O, GC_for_non_fuel_work_O, 0.053 * AADT__B_dict_low[year], 0.053 * AADT__B_O_dict_low[year], 0.053 * AADT__O_dict_low[year])
    N_work_B_high[year] = get_ROH(GC_for_non_fuel_work_B, GC_for_non_fuel_work_B_O, GC_for_non_fuel_work_O, 0.053 * AADT__B_dict_high[year], 0.053 * AADT__B_O_dict_high[year], 0.053 * AADT__O_dict_high[year])
    F_non_work_A_low[year] = get_ROH(GC_for_fuel_cost_A[year], GC_for_fuel_cost_O[year], GC_for_fuel_cost_O[year], 0.947 * AADT__A_dict_low[year], 0.947 * AADT__A_O_dict_low[year], 0.947 * AADT__O_dict_low[year])
    F_non_work_A_high[year] = get_ROH(GC_for_fuel_cost_A[year], GC_for_fuel_cost_O[year], GC_for_fuel_cost_O[year], 0.947 * AADT__A_dict_high[year], 0.947 * AADT__A_O_dict_high[year], 0.947 * AADT__O_dict_high[year])
    F_non_work_B_low[year] = get_ROH(GC_for_fuel_cost_B[year], GC_for_fuel_cost_O[year], GC_for_fuel_cost_O[year], 0.947 * AADT__B_dict_low[year], 0.947 * AADT__B_O_dict_low[year], 0.947 * AADT__O_dict_low[year])
    F_non_work_B_high[year] = get_ROH(GC_for_fuel_cost_B[year], GC_for_fuel_cost_O[year], GC_for_fuel_cost_O[year], 0.947 * AADT__B_dict_high[year], 0.947 * AADT__B_O_dict_high[year], 0.947 * AADT__O_dict_high[year])
    N_non_work_A_low[year] = get_ROH(GC_for_non_fuel_non_Work_A, GC_for_non_fuel_non_Work_A_O, GC_for_non_fuel_non_Work_O, 0.947 * AADT__A_dict_low[year], 0.947 * AADT__A_O_dict_low[year], 0.947 * AADT__O_dict_low[year])
    N_non_work_A_high[year] = get_ROH(GC_for_non_fuel_non_Work_A, GC_for_non_fuel_non_Work_A_O, GC_for_non_fuel_non_Work_O, 0.947 * AADT__A_dict_high[year], 0.947 * AADT__A_O_dict_high[year], 0.947 * AADT__O_dict_high[year])
    N_non_work_B_low[year] = get_ROH(GC_for_non_fuel_non_Work_B, GC_for_non_fuel_non_Work_B_O, GC_for_non_fuel_non_Work_O, 0.947 * AADT__B_dict_low[year], 0.947 * AADT__B_O_dict_low[year], 0.947 * AADT__O_dict_low[year])
    N_non_work_B_high[year] = get_ROH(GC_for_non_fuel_non_Work_B, GC_for_non_fuel_non_Work_B_O, GC_for_non_fuel_non_Work_O, 0.947 * AADT__B_dict_high[year], 0.947 * AADT__B_O_dict_high[year], 0.947 * AADT__O_dict_high[year])


In [1175]:
# 计算间接税
indirect_tax_A_low = {}
indirect_tax_A_high = {}
indirect_tax_B_low = {}
indirect_tax_B_high = {}
for year in range(2031, 2091):
    indirect_tax_A_low[year] = calculate_tax(1, F_work_A_low[year], N_work_A_low[year]) + calculate_tax(0, F_non_work_A_low[year], N_non_work_A_low[year])
    indirect_tax_A_high[year] = calculate_tax(1, F_work_A_high[year], N_work_A_high[year]) + calculate_tax(0, F_non_work_A_high[year], N_non_work_A_high[year])
    indirect_tax_B_low[year] = calculate_tax(1, F_work_B_low[year], N_work_B_low[year]) + calculate_tax(0, F_non_work_B_low[year], N_non_work_B_low[year])
    indirect_tax_B_high[year] = calculate_tax(1, F_work_B_high[year], N_work_B_high[year]) + calculate_tax(0, F_non_work_B_high[year], N_non_work_B_high[year])


下一步考虑温室气体效应，选用市场价格，仍然使用半数法算出效益；其实这里算作负效应还是正的支出，是一个值得讨论的话题；
假如改成汽油车，这一项估计要爆表了……

In [1176]:

GC_emission_A = {}
GC_emission_B = {}
GC_emission_O = {}
for year in range(2031, 2091):
    GC_emission_A[year] = emission_factor[year] * money_value_of_emission[year] * fuel_effieciency[year] * road_length__A * 1.5893 / 1000
    GC_emission_B[year] = emission_factor[year] * money_value_of_emission[year] * fuel_effieciency[year] * road_length_B * 1.5893 / 1000
    GC_emission_O[year] = emission_factor[year] * money_value_of_emission[year] * fuel_effieciency[year] * road_length__O * 1.5893 / 1000

user_benefit_A_emission_low = {}
user_benefit_A_emission_high = {}
user_benefit_B_emission_low = {}
user_benefit_B_emission_high = {}
for year in range(2031, 2091):
    user_benefit_A_emission_low[year] = get_ROH(GC_emission_A[year], GC_emission_O[year], GC_emission_O[year],  AADT__A_dict_low[year],  AADT__A_O_dict_low[year],  AADT__O_dict_low[year])
    user_benefit_A_emission_high[year] = get_ROH(GC_emission_A[year], GC_emission_O[year], GC_emission_O[year],  AADT__A_dict_high[year],  AADT__A_O_dict_high[year],  AADT__O_dict_high[year])
    user_benefit_B_emission_low[year] = get_ROH(GC_emission_B[year], GC_emission_O[year], GC_emission_O[year],  AADT__B_dict_low[year],  AADT__B_O_dict_low[year],  AADT__O_dict_low[year])
    user_benefit_B_emission_high[year] = get_ROH(GC_emission_B[year], GC_emission_O[year], GC_emission_O[year],  AADT__B_dict_high[year],  AADT__B_O_dict_high[year],  AADT__O_dict_high[year])





最后一步，成本（建设和维护），我决定上面的都算是收益，哪怕是负收益

In [1177]:
maintainance_cost_A_2030 = maintainance_cost_A * 1.5893
maintainance_cost_B_2030 = maintenance_cost__B * 1.5893
maintainance_cost_A_dict = {}
maintainance_cost_B_dict = {}
for year in range(2031, 2091):
    maintainance_cost_A_dict[year] = maintainance_cost_A_2030
    maintainance_cost_B_dict[year] = maintainance_cost_B_2030

construction_cost__A_modified = construction_cost__A /2 * (1 + 1.035) * 1.5893
construction_cost__B_modified = construction_cost__B /2 * (1 + 1.035) * 1.5893



整合各部分成本和收益

In [1178]:
'''
1. 用半数法的部分
benefit_travel_time ... GC_for_journey_time
benefit_vehicle_operating_cost ... GC_for_fuel_cost, GC_for_non_fuel_work
benefit_emission ... GC_emission
GC_total_A_low = GC_for_journey_time_A + GC_for_fuel_cost_A + GC_emission_A + 0.053 * GC_for_non_fuel_non_work_A

2. 用复杂公式的部分，也用了半数法
benefit_indirect_tax ... indirect_tax_A, indirect_tax_B
3. 用净收益的部分
non_fuel_non_work ... GC_for_non_fuel_non_Work
'''
benefit_A_low = discount_to_2030(user_benefit_A_work_low, 2090) + discount_to_2030(user_benefit_A_non_work_low, 2090) + discount_to_2030(user_benefit_non_work_non_fuel_A_low, 2090) + discount_to_2030(user_benefit_A_emission_low, 2090) - discount_to_2030(indirect_tax_A_low, 2090)
benefit_A_high = discount_to_2030(user_benefit_A_work_high, 2090) + discount_to_2030(user_benefit_A_non_work_high, 2090) + discount_to_2030(user_benefit_non_work_non_fuel_A_high, 2090) + discount_to_2030(user_benefit_A_emission_high, 2090) - discount_to_2030(indirect_tax_A_high, 2090)
cost_A = construction_cost__A_modified + discount_to_2030(maintainance_cost_A_dict, 2090)
benefit_B_low = discount_to_2030(user_benefit_B_work_low, 2090) + discount_to_2030(user_benefit_B_non_work_low, 2090) + discount_to_2030(user_benefit_non_work_non_fuel_B_low, 2090) + discount_to_2030(user_benefit_B_emission_low, 2090) - discount_to_2030(indirect_tax_B_low, 2090)
benefit_B_high = discount_to_2030(user_benefit_B_work_high, 2090) + discount_to_2030(user_benefit_B_non_work_high, 2090) + discount_to_2030(user_benefit_non_work_non_fuel_B_high, 2090) + discount_to_2030(user_benefit_B_emission_high, 2090) - discount_to_2030(indirect_tax_B_high, 2090)
cost_B = construction_cost__B_modified + discount_to_2030(maintainance_cost_B_dict, 2090)
benefits_A_low = {}
benefits_A_high = {}
benefits_B_low = {}
benefits_B_high = {}
for year in range(2031, 2091):
    benefits_A_low[year] = discount_to_2030(user_benefit_A_work_low, year) + discount_to_2030(user_benefit_A_non_work_low, year) + discount_to_2030(user_benefit_non_work_non_fuel_A_low, year) + discount_to_2030(user_benefit_A_emission_low, year) - discount_to_2030(indirect_tax_A_low, year)
    benefits_A_high[year] = discount_to_2030(user_benefit_A_work_high, year) + discount_to_2030(user_benefit_A_non_work_high, year) + discount_to_2030(user_benefit_non_work_non_fuel_A_high, year) + discount_to_2030(user_benefit_A_emission_high, year) - discount_to_2030(indirect_tax_A_high, year)
    benefits_B_low[year] = discount_to_2030(user_benefit_B_work_low, year) + discount_to_2030(user_benefit_B_non_work_low, year) + discount_to_2030(user_benefit_non_work_non_fuel_B_low, year) + discount_to_2030(user_benefit_B_emission_low, year) - discount_to_2030(indirect_tax_B_low, year)
    benefits_B_high[year] = discount_to_2030(user_benefit_B_work_high, year) + discount_to_2030(user_benefit_B_non_work_high, year) + discount_to_2030(user_benefit_non_work_non_fuel_B_high, year) + discount_to_2030(user_benefit_B_emission_high, year) - discount_to_2030(indirect_tax_B_high, year)
    


PVB, PVC

In [1179]:
PVB_A_low = benefit_A_low
PVB_A_high = benefit_A_high
PVB_B_low = benefit_B_low
PVB_B_high = benefit_B_high
PVC_A_low = cost_A
PVC_A_high = cost_A
PVC_B_low = cost_B
PVC_B_high = cost_B

print(f"{'PVB_A_low:':>15}{PVB_A_low:>15.2f}")
print(f"{'PVB_A_high:':>15}{PVB_A_high:>15.2f}")
print(f"{'PVB_B_low:':>15}{PVB_B_low:>15.2f}")
print(f"{'PVB_B_high:':>15}{PVB_B_high:>15.2f}")
print(f"{'PVC_A_low:':>15}{PVC_A_low:>15.2f}")
print(f"{'PVC_A_high:':>15}{PVC_A_high:>15.2f}")
print(f"{'PVC_B_low:':>15}{PVC_B_low:>15.2f}")
print(f"{'PVC_B_high:':>15}{PVC_B_high:>15.2f}")




     PVB_A_low:   369978597.76
    PVB_A_high:   431038028.52
     PVB_B_low:   237714693.29
    PVB_B_high:   277405907.31
     PVC_A_low:   145943436.50
    PVC_A_high:   145943436.50
     PVC_B_low:   124920970.75
    PVC_B_high:   124920970.75


NPV

In [1180]:
NPV_A_low = benefit_A_low - cost_A
NPV_A_high = benefit_A_high - cost_A
NPV_B_low = benefit_B_low - cost_B
NPV_B_high = benefit_B_high - cost_B
# 格式化构成结果的各项和结果,数值靠右对齐
print(f"{'NPV_A_low:':>15}{NPV_A_low:>15.2f}")
print(f"{'NPV_A_high:':>15}{NPV_A_high:>15.2f}")
print(f"{'NPV_B_low:':>15}{NPV_B_low:>15.2f}")
print(f"{'NPV_B_high:':>15}{NPV_B_high:>15.2f}")


     NPV_A_low:   224035161.26
    NPV_A_high:   285094592.02
     NPV_B_low:   112793722.54
    NPV_B_high:   152484936.56


BCR

In [1181]:
BCR_A_low = benefit_A_low / cost_A
BCR_A_high = benefit_A_high / cost_A
BCR_B_low = benefit_B_low / cost_B
BCR_B_high = benefit_B_high / cost_B
print(f"{'BCR_A_low:':>15} {BCR_A_low:>15.2f}")
print(f"{'BCR_A_high:':>15} {BCR_A_high:>15.2f}")
print(f"{'BCR_B_low:':>15} {BCR_B_low:>15.2f}")
print(f"{'BCR_B_high:':>15} {BCR_B_high:>15.2f}")

     BCR_A_low:            2.54
    BCR_A_high:            2.95
     BCR_B_low:            1.90
    BCR_B_high:            2.22


IRR，使用numpy_financial包

In [1182]:
import numpy_financial as npf

cash_flows_A_low = [-construction_cost__A] + [benefits_A_low[year] - maintainance_cost_A_dict[year] for year in range(2031, 2091)]
cash_flows_A_high = [-construction_cost__A] + [benefits_A_high[year] - maintainance_cost_A_dict[year] for year in range(2031, 2091)]
cash_flows_B_low = [-construction_cost__B] + [benefits_B_low[year] - maintainance_cost_B_dict[year] for year in range(2031, 2091)]
cash_flows_B_high = [-construction_cost__B] + [benefits_B_high[year] - maintainance_cost_B_dict[year] for year in range(2031, 2091)]

irr_flows_A_low = npf.irr(cash_flows_A_low)
irr_flows_A_high = npf.irr(cash_flows_A_high)
irr_flows_B_low = npf.irr(cash_flows_B_low)
irr_flows_B_high = npf.irr(cash_flows_B_high)

print(f"{'IRR_A_low:':>15} {irr_flows_A_low:>15.2%}")
print(f"{'IRR_A_high:':>15} {irr_flows_A_high:>15.2%}")
print(f"{'IRR_B_low:':>15} {irr_flows_B_low:>15.2%}")
print(f"{'IRR_B_high:':>15} {irr_flows_B_high:>15.2%}")


     IRR_A_low:          34.86%
    IRR_A_high:          35.50%
     IRR_B_low:          28.43%
    IRR_B_high:          29.03%


Payback_Period

In [1183]:
def calculate_Payback_Period(benefits, costs, constant_cost):
    for year in range(2031, 2091):
        total_benefit = discount_to_2030(benefits, year)
        total_cost = discount_to_2030(costs, year) + constant_cost
        if total_benefit > total_cost:
            return year - 1

Payback_Period_A_low = calculate_Payback_Period(benefits_A_low, maintainance_cost_A_dict, construction_cost__A)
Payback_Period_A_high = calculate_Payback_Period(benefits_A_high, maintainance_cost_A_dict, construction_cost__A)
Payback_Period_B_low = calculate_Payback_Period(benefits_B_low, maintainance_cost_B_dict, construction_cost__B)
Payback_Period_B_high = calculate_Payback_Period(benefits_B_high, maintainance_cost_B_dict, construction_cost__B)
print('Payback_Period_A_low:', Payback_Period_A_low)
print('Payback_Period_A_high:', Payback_Period_A_high)
print('Payback_Period_B_low:', Payback_Period_B_low)
print('Payback_Period_B_high:', Payback_Period_B_high)

Payback_Period_A_low: 2034
Payback_Period_A_high: 2034
Payback_Period_B_low: 2035
Payback_Period_B_high: 2035


FYRR 


In [1184]:
benefit_A_low_2031 = benefits_A_low[2031]
benefit_A_high_2031 = benefits_A_high[2031]
benefit_B_low_2031 = benefits_B_low[2031]
benefit_B_high_2031 = benefits_B_high[2031]
cost_A_2031 = maintainance_cost_A_dict[2031] 
cost_B_2031 = maintainance_cost_B_dict[2031]
initial_cost_A = construction_cost__A_modified
initial_cost_B = construction_cost__B_modified
fyrr_A_low = (benefit_A_low_2031 - cost_A_2031) / initial_cost_A
fyrr_A_high = (benefit_A_high_2031 - cost_A_2031) / initial_cost_A
fyrr_B_low = (benefit_B_low_2031 - cost_B_2031) / initial_cost_B
fyrr_B_high = (benefit_B_high_2031 - cost_B_2031) / initial_cost_B
print('FYRR_A_low:', fyrr_A_low)
print('FYRR_A_high:', fyrr_A_high)
print('FYRR_B_low:', fyrr_B_low)
print('FYRR_B_high:', fyrr_B_high)


FYRR_A_low: 0.056554860246494934
FYRR_A_high: 0.05677305211602514
FYRR_B_low: 0.03876813046414885
FYRR_B_high: 0.03888857389519462


现在做一个总的汇总表，包含题目要求的各量：

1. PVC
2. PVB
3. NPV
4. BCR
5. IRR
6. FYRR
7. Pay_back_Period


In [1185]:
'''
现在做一个总的汇总表，包含题目要求的各量：

1. PVC
2. PVB
3. NPV
4. BCR
5. IRR
6. FYRR
7. Pay_back_Period
'''
import pandas as pd

data = {
    'A_low': [PVC_A_low, PVB_A_low, NPV_A_low, BCR_A_low, irr_flows_A_low, Payback_Period_A_low, fyrr_A_low],
    'A_high': [PVC_A_high, PVB_A_high, NPV_A_high, BCR_A_high, irr_flows_A_high, Payback_Period_A_high, fyrr_A_high],
    'B_low': [PVC_B_low, PVB_B_low, NPV_B_low, BCR_B_low, irr_flows_B_low, Payback_Period_B_low, fyrr_B_low],
    'B_high': [PVC_B_high, PVB_B_high, NPV_B_high, BCR_B_high, irr_flows_B_high, Payback_Period_B_high, fyrr_B_high]
}

df = pd.DataFrame(data, index=['PVC', 'PVB', 'NPV', 'BCR', 'IRR', 'Payback_Period', 'FYRR'])
print(df)
with pd.ExcelWriter('output_sum.xlsx') as writer:
    df.to_excel(writer, sheet_name='Sheet1')

                       A_low        A_high         B_low        B_high
PVC             1.459434e+08  1.459434e+08  1.249210e+08  1.249210e+08
PVB             3.699786e+08  4.310380e+08  2.377147e+08  2.774059e+08
NPV             2.240352e+08  2.850946e+08  1.127937e+08  1.524849e+08
BCR             2.535082e+00  2.953459e+00  1.902921e+00  2.220651e+00
IRR             3.486231e-01  3.550432e-01  2.842724e-01  2.903446e-01
Payback_Period  2.034000e+03  2.034000e+03  2.035000e+03  2.035000e+03
FYRR            5.655486e-02  5.677305e-02  3.876813e-02  3.888857e-02


现在做一个汇总表，因为如果表格太大的话不好放入到文档中，因此，分几部分来输出:

1. VOC Benefit Data (这里包含了旅行时间，燃料成本和非燃料成本)
2. Emission Benefit Data
3. Indirect Tax Benefit Data
4. Cost Data

In [1186]:
import pandas as pd

# 初始化DataFrame
years = sorted(list(benefits_A_low.keys()))
df = pd.DataFrame(index=years)

# 用户利益数据（避免重复赋值，直接在创建时填充）
df_benefit_VOC = pd.DataFrame(index=df.index)
df_benefit_VOC['VOC_benefit_A_work_low'] = df.index.map(user_benefit_A_work_low)
df_benefit_VOC['VOC_benefit_A_work_high'] = df.index.map(user_benefit_A_work_high)
df_benefit_VOC['VOC_benefit_B_work_low'] = df.index.map(user_benefit_B_work_low)
df_benefit_VOC['VOC_benefit_B_work_high'] = df.index.map(user_benefit_B_work_high)
df_benefit_VOC['VOC_benefit_A_non_work_low'] = df.index.map(user_benefit_A_non_work_low)
df_benefit_VOC['VOC_benefit_A_non_work_high'] = df.index.map(user_benefit_A_non_work_high)
df_benefit_VOC['VOC_benefit_B_non_work_low'] = df.index.map(user_benefit_B_non_work_low)
df_benefit_VOC['VOC_benefit_B_non_work_high'] = df.index.map(user_benefit_B_non_work_high)
df_benefit_VOC['VOC_beneift_non_fuel_non_work_A_low'] = df.index.map(user_benefit_non_work_non_fuel_A_low)
df_benefit_VOC['VOC_beneift_non_fuel_non_work_A_high'] = df.index.map(user_benefit_non_work_non_fuel_A_high)
df_benefit_VOC['VOC_beneift_non_fuel_non_work_B_low'] = df.index.map(user_benefit_non_work_non_fuel_B_low)
df_benefit_VOC['VOC_beneift_non_fuel_non_work_B_high'] = df.index.map(user_benefit_non_work_non_fuel_B_high)
df_benefit_emission = pd.DataFrame(index=df.index)
df_benefit_emission['emission_benefit_A_low'] = df.index.map(user_benefit_A_emission_low)
df_benefit_emission['emission_benefit_A_high'] = df.index.map(user_benefit_A_emission_high)
df_benefit_emission['emission_benefit_B_low'] = df.index.map(user_benefit_B_emission_low)
df_benefit_emission['emission_benefit_B_high'] = df.index.map(user_benefit_B_emission_high)
df_benefit_indirect_tax = pd.DataFrame(index=df.index)
df_benefit_indirect_tax['indirect_tax_benefit_A_low'] = df.index.map(indirect_tax_A_low)
df_benefit_indirect_tax['indirect_tax_benefit_A_high'] = df.index.map(indirect_tax_A_high)
df_benefit_indirect_tax['indirect_tax_benefit_B_low'] = df.index.map(indirect_tax_B_low)
df_benefit_indirect_tax['indirect_tax_benefit_B_high'] = df.index.map(indirect_tax_B_high)


# 其他数据可以以类似的方式创建专门的DataFrame
df_cost = pd.DataFrame(index=df.index)
df_cost['construction_cost__A'] = construction_cost__A
df_cost['construction_cost__B'] = construction_cost__B 
df_cost['maintainance_cost_A'] = df.index.map(maintainance_cost_A_dict)
df_cost['maintainance_cost_B'] = df.index.map(maintainance_cost_B_dict)


# 注意：每个专门的DataFrame（例如df_AADT, df_GC, df_benefit等）可以根据需求单独保存或处理





In [1187]:
# Excel文件路径
excel_path = 'output_data.xlsx'

# 使用ExcelWriter将DataFrame写入不同的工作表
with pd.ExcelWriter(excel_path, engine='openpyxl', mode='w') as writer:
    df_benefit_VOC.to_excel(writer, sheet_name='VOC Benefit Data', index=True)
    df_benefit_emission.to_excel(writer, sheet_name='Emission Benefit Data', index=True)
    df_benefit_indirect_tax.to_excel(writer, sheet_name='Indirect Tax Benefit Data', index=True)
    df_cost.to_excel(writer, sheet_name='Cost Data', index=True)

   


调整下列宽

In [1188]:
import pandas as pd
from openpyxl.utils import get_column_letter

excel_path = 'output_data.xlsx'
with pd.ExcelWriter(excel_path, engine='openpyxl', mode='w') as writer:
    df_benefit_VOC.to_excel(writer, sheet_name='VOC Benefit Data', index=True)
    df_benefit_emission.to_excel(writer, sheet_name='Emission Benefit Data', index=True)
    df_benefit_indirect_tax.to_excel(writer, sheet_name='Indirect Tax Benefit Data', index=True)
    df_cost.to_excel(writer, sheet_name='Cost Data', index=True)

    workbook = writer.book
    worksheet = workbook['VOC Benefit Data']
    for col in worksheet.columns:
        worksheet.column_dimensions[get_column_letter(col[0].column)].width = 40
    worksheet = workbook['Emission Benefit Data']
    for col in worksheet.columns:
        worksheet.column_dimensions[get_column_letter(col[0].column)].width = 40
    worksheet = workbook['Indirect Tax Benefit Data']
    for col in worksheet.columns:
        worksheet.column_dimensions[get_column_letter(col[0].column)].width = 40
    worksheet = workbook['Cost Data']
    for col in worksheet.columns:
        worksheet.column_dimensions[get_column_letter(col[0].column)].width = 40

下面对construction cost, time value, emission, life span, discount进行敏感性测试

我个人认为该商业计划是一个OBC（Outline Business Case），这只是一个主观判断，因此 optimism_bias_uplifts 取值为 23%

根据说明，计算BCR即可，因此，在使用乐观偏差调整后得出BCR

同时，因为ppt中make a decision 在敏感性测试之后，所以测试内容还是得包括方案B....

先计算construction_cost

In [1189]:
optimism_bias_uplifts = 1.23
cost_A_uplift = construction_cost__A_modified * optimism_bias_uplifts + discount_to_2030(maintainance_cost_A_dict, 2090)
cost_B_uplift = construction_cost__B_modified * optimism_bias_uplifts + discount_to_2030(maintainance_cost_B_dict, 2090)

def BCR_calculator_by_cost_uplift(benefit, construction_cost, maintainance_cost, optimism_bias_uplifts):
    cost_uplift =   construction_cost * optimism_bias_uplifts + maintainance_cost
    return benefit / cost_uplift

'''
BCR_A_low = benefit_A_low / cost_A
BCR_A_high = benefit_A_high / cost_A
BCR_B_low = benefit_B_low / cost_B
BCR_B_high = benefit_B_high / cost_B
'''

BCR_A_low_uplift = BCR_calculator_by_cost_uplift(benefit_A_low, construction_cost__A_modified, discount_to_2030(maintainance_cost_A_dict, 2090), optimism_bias_uplifts)
BCR_A_high_uplift = BCR_calculator_by_cost_uplift(benefit_A_high, construction_cost__A_modified, discount_to_2030(maintainance_cost_A_dict, 2090), optimism_bias_uplifts)
BCR_B_low_uplift = BCR_calculator_by_cost_uplift(benefit_B_low, construction_cost__B_modified, discount_to_2030(maintainance_cost_B_dict, 2090), optimism_bias_uplifts)
BCR_B_high_uplift = BCR_calculator_by_cost_uplift(benefit_B_high, construction_cost__B_modified, discount_to_2030(maintainance_cost_B_dict, 2090), optimism_bias_uplifts)
print(f"{'BCR_A_low_uplift:':>15} {BCR_A_low_uplift:>15.2f}")
print(f"{'BCR_A_high_uplift:':>15} {BCR_A_high_uplift:>15.2f}")
print(f"{'BCR_B_low_uplift:':>15} {BCR_B_low_uplift:>15.2f}")
print(f"{'BCR_B_high_uplift:':>15} {BCR_B_high_uplift:>15.2f}")

def cost_needed_for_BCR(benefit, bcr, cost_uplift):
    """计算达到特定BCR所需的成本。"""
    return benefit / bcr - cost_uplift

def overrun_needed_for_BCR(benefit, bcr, cost):
    """计算达到特定BCR所需的成本超支比例。"""
    return benefit / bcr / cost - 1

cost_needed_for_BCR_2p5_low = cost_needed_for_BCR(benefit_A_low, 2.5, cost_A_uplift)
cost_needed_for_BCR_2p5_high = cost_needed_for_BCR(benefit_A_high, 2.5, cost_A_uplift)
cost_needed_for_BCR_2_low = cost_needed_for_BCR(benefit_A_low, 2, cost_A_uplift)
cost_needed_for_BCR_2_high = cost_needed_for_BCR(benefit_A_high, 2, cost_A_uplift)

cost_needed_for_B_for_BCR_2_low = cost_needed_for_BCR(benefit_B_low, 2, cost_B_uplift)
cost_needed_for_B_for_BCR_2_high = cost_needed_for_BCR(benefit_B_high, 2, cost_B_uplift)
cost_needed_for_B_for_BCR_1p5_low = cost_needed_for_BCR(benefit_B_low, 1.5, cost_B_uplift)
cost_needed_for_B_for_BCR_1p5_high = cost_needed_for_BCR(benefit_B_high, 1.5, cost_B_uplift)

overrun_needed_for_BCR_2p5_low = overrun_needed_for_BCR(benefit_A_low, 2.5, cost_A)
overrun_needed_for_BCR_2p5_high = overrun_needed_for_BCR(benefit_A_high, 2.5, cost_A)
overrun_needed_for_BCR_2_low = overrun_needed_for_BCR(benefit_A_low, 2, cost_A)
overrun_needed_for_BCR_2_high = overrun_needed_for_BCR(benefit_A_high, 2, cost_A)

overrun_needed_for_B_for_BCR_2_low = overrun_needed_for_BCR(benefit_B_low, 2, cost_B)
overrun_needed_for_B_for_BCR_2_high = overrun_needed_for_BCR(benefit_B_high, 2, cost_B)
overrun_needed_for_B_for_BCR_1p5_low = overrun_needed_for_BCR(benefit_B_low, 1.5, cost_B)
overrun_needed_for_B_for_BCR_1p5_high = overrun_needed_for_BCR(benefit_B_high, 1.5, cost_B)


vfm_data_1 = {
    "A_low: ": [cost_A, cost_A_uplift, benefit_A_low, BCR_A_low_uplift, cost_needed_for_BCR_2p5_low, cost_needed_for_BCR_2_low, overrun_needed_for_BCR_2p5_low, overrun_needed_for_BCR_2_low],
    "A_high: ": [cost_A, cost_A_uplift, benefit_A_high, BCR_A_high_uplift, cost_needed_for_BCR_2p5_high, cost_needed_for_BCR_2_high, overrun_needed_for_BCR_2p5_high, overrun_needed_for_BCR_2_high],
    "B_low:": [cost_B, cost_B_uplift, benefit_B_low, BCR_B_low_uplift, cost_needed_for_B_for_BCR_2_low, cost_needed_for_B_for_BCR_1p5_low, overrun_needed_for_B_for_BCR_2_low, overrun_needed_for_B_for_BCR_1p5_low],
    "B_high:": [cost_B, cost_B_uplift, benefit_B_high, BCR_B_high_uplift, cost_needed_for_B_for_BCR_2_high, cost_needed_for_B_for_BCR_1p5_high, overrun_needed_for_B_for_BCR_2_high, overrun_needed_for_B_for_BCR_1p5_high]
}

df = pd.DataFrame(vfm_data_1, index=['PVC', 'PVC_uplift', 'PVB', 'BCR_uplift', 'cost_needed_for_BCR_high', 'cost_needed_for_BCR_low', 'overrun_needed_for_BCR_high', 'overrun_needed_for_BCR_low'])
print(df)



BCR_A_low_uplift:            2.06
BCR_A_high_uplift:            2.40
BCR_B_low_uplift:            1.55
BCR_B_high_uplift:            1.81
                                  A_low:       A_high:         B_low:  \
PVC                          1.459434e+08  1.459434e+08  1.249210e+08   
PVC_uplift                   1.794177e+08  1.794177e+08  1.535600e+08   
PVB                          3.699786e+08  4.310380e+08  2.377147e+08   
BCR_uplift                   2.062108e+00  2.402428e+00  1.548024e+00   
cost_needed_for_BCR_high    -3.142623e+07 -7.002459e+06 -3.470269e+07   
cost_needed_for_BCR_low      5.571628e+06  3.610134e+07  4.916425e+06   
overrun_needed_for_BCR_high  1.403285e-02  1.813838e-01 -4.853968e-02   
overrun_needed_for_BCR_low   2.675411e-01  4.767297e-01  2.686138e-01   

                                  B_high:  
PVC                          1.249210e+08  
PVC_uplift                   1.535600e+08  
PVB                          2.774059e+08  
BCR_uplift                  

用非常牛逼的一个开源软件，[webplotdigitizer](https://automeris.io/documentation.html),生成了对应曲线提取的数据点，根据这些数据点线性插值

> 这个步骤是模仿tag 1.2教程中的示范，使用的是vfm评价图表

In [1190]:
import numpy as np

def linear_interpolation(x_values, y_values, target_y):
    """
    Performs linear interpolation to find the corresponding x value for a given target y value.

    Parameters:
    - x_values (list or array): The x values of the curve.
    - y_values (list or array): The y values of the curve.
    - target_y (float): The y value for which we want to find the corresponding x value.

    Returns:
    - target_x (float): The interpolated x value corresponding to the target y value.
    """
    if not (len(x_values) == len(y_values)):
        raise ValueError("x_values and y_values must have the same length")

    # Ensuring x_values and y_values are numpy arrays for efficient computation
    x_values = np.array(x_values)
    y_values = np.array(y_values)

    # Finding the first instance where the y value in the array exceeds the target y value
    for i in range(len(y_values) - 1):
        if y_values[i] <= target_y <= y_values[i + 1] or y_values[i] >= target_y >= y_values[i + 1]:
            # Performing linear interpolation
            slope = (x_values[i + 1] - x_values[i]) / (y_values[i + 1] - y_values[i])
            return x_values[i] + slope * (target_y - y_values[i])

    raise ValueError("target_y is outside the range of y_values")

# Reload the CSV with correct column interpretation
df = pd.read_csv('OBC_cost_road.csv', header=None, names=['x', 'y'])

# Convert percentage y values to decimals for interpolation
# -2对应的是高值，-1对应的是低值
y_target_values = [value[-2] for key, value in vfm_data_1.items()] + [value[-1] for key, value in vfm_data_1.items()]


# Use the linear interpolation function to find corresponding x values for the given y targets
x_interpolated = [linear_interpolation(df['x'].values, df['y'].values, y) for y in y_target_values]
x_interpolated


[0.1916910647396802,
 0.4446729687018592,
 0.13540170916115274,
 0.3447980155238627,
 0.6430212909149717,
 0.7641454680750114,
 0.6432984600157473,
 0.766059801289206]

继续计算关于项目周期的计算

In [1191]:

life_span = 60
life_span_corrected = int(60 / optimism_bias_uplifts)
life_span_corrected += 2030
print(life_span_corrected)
benefit_A_low_l_corrected = discount_to_2030(user_benefit_A_work_low, life_span_corrected) + discount_to_2030(user_benefit_A_non_work_low, life_span_corrected) + discount_to_2030(user_benefit_non_work_non_fuel_A_low, life_span_corrected) + discount_to_2030(user_benefit_A_emission_low, life_span_corrected) - discount_to_2030(indirect_tax_A_low, life_span_corrected)
benefit_A_high_l_corrected = discount_to_2030(user_benefit_A_work_high, life_span_corrected) + discount_to_2030(user_benefit_A_non_work_high, life_span_corrected) + discount_to_2030(user_benefit_non_work_non_fuel_A_high, life_span_corrected) + discount_to_2030(user_benefit_A_emission_high, life_span_corrected) - discount_to_2030(indirect_tax_A_high, life_span_corrected)
cost_A_l_corrected = construction_cost__A_modified + discount_to_2030(maintainance_cost_A_dict, life_span_corrected)
benefit_B_low_l_corrected = discount_to_2030(user_benefit_B_work_low, life_span_corrected) + discount_to_2030(user_benefit_B_non_work_low, life_span_corrected) + discount_to_2030(user_benefit_non_work_non_fuel_B_low, life_span_corrected) + discount_to_2030(user_benefit_B_emission_low, life_span_corrected) - discount_to_2030(indirect_tax_B_low, life_span_corrected)
benefit_B_high_l_corrected = discount_to_2030(user_benefit_B_work_high, life_span_corrected) + discount_to_2030(user_benefit_B_non_work_high, life_span_corrected) + discount_to_2030(user_benefit_non_work_non_fuel_B_high, life_span_corrected) + discount_to_2030(user_benefit_B_emission_high, life_span_corrected) - discount_to_2030(indirect_tax_B_high, life_span_corrected)
cost_B_l_corrected = construction_cost__B_modified + discount_to_2030(maintainance_cost_B_dict, life_span_corrected)

BCR_A_low_l_corrected = benefit_A_low_l_corrected / cost_A_l_corrected
BCR_A_high_l_corrected = benefit_A_high_l_corrected / cost_A_l_corrected
BCR_B_low_l_corrected = benefit_B_low_l_corrected / cost_B_l_corrected
BCR_B_high_l_corrected = benefit_B_high_l_corrected / cost_B_l_corrected
print(f"{'BCR_A_low_l_corrected:':>15} {BCR_A_low_l_corrected:>15.2f}")
print(f"{'BCR_A_high_l_corrected:':>15} {BCR_A_high_l_corrected:>15.2f}")
print(f"{'BCR_B_low_l_corrected:':>15} {BCR_B_low_l_corrected:>15.2f}")
print(f"{'BCR_B_high_l_corrected:':>15} {BCR_B_high_l_corrected:>15.2f}")
    



2078
BCR_A_low_l_corrected:            2.18
BCR_A_high_l_corrected:            2.52
BCR_B_low_l_corrected:            1.62
BCR_B_high_l_corrected:            1.88


In [1192]:
'''
废弃掉的代码，因为由于成本和收益都在变化，所以这种计算方法失去实际的意义
cost_needed_for_BCR_2p5_low = cost_needed_for_BCR(benefit_A_low_l_corrected, 2.5, cost_A_l_corrected)
cost_needed_for_BCR_3_high = cost_needed_for_BCR(benefit_A_high_l_corrected, 3, cost_A_l_corrected)
cost_needed_for_BCR_2_low = cost_needed_for_BCR(benefit_A_low_l_corrected, 2, cost_A_l_corrected)
cost_needed_for_BCR_2p5_high = cost_needed_for_BCR(benefit_A_high_l_corrected, 2.5, cost_A_l_corrected)

cost_needed_for_B_for_BCR_2_low = cost_needed_for_BCR(benefit_B_low_l_corrected, 2, cost_B_l_corrected)
cost_needed_for_B_for_BCR_1p5_low = cost_needed_for_BCR(benefit_B_low_l_corrected, 1.5, cost_B_l_corrected)
cost_needed_for_B_for_BCR_2_high = cost_needed_for_BCR(benefit_B_high_l_corrected, 2, cost_B_l_corrected)
cost_needed_for_B_for_BCR_1p5_high = cost_needed_for_BCR(benefit_B_high_l_corrected, 1.5, cost_B_l_corrected)

def overrun_needed_for_BCR(benefit_corrected, cost_need, cost):
    return (benefit_corrected + cost_need) / cost - 1

overrun_needed_for_BCR_2p5_low = overrun_needed_for_BCR(benefit_A_low_l_corrected, cost_needed_for_BCR_2p5_low, cost_A)
overrun_needed_for_BCR_3_high = overrun_needed_for_BCR(benefit_A_high_l_corrected, cost_needed_for_BCR_3_high, cost_A)
overrun_needed_for_BCR_2_low = overrun_needed_for_BCR(benefit_A_low_l_corrected, cost_needed_for_BCR_2_low, cost_A)
overrun_needed_for_BCR_2p5_high = overrun_needed_for_BCR(benefit_A_high_l_corrected, cost_needed_for_BCR_2p5_high, cost_A)

overrun_needed_for_B_for_BCR_2_low = overrun_needed_for_BCR(benefit_B_low_l_corrected, cost_needed_for_B_for_BCR_2_low, cost_B)
overrun_needed_for_B_for_BCR_1p5_low = overrun_needed_for_BCR(benefit_B_low_l_corrected, cost_needed_for_B_for_BCR_1p5_low, cost_B)
overrun_needed_for_B_for_BCR_2_high = overrun_needed_for_BCR(benefit_B_high_l_corrected, cost_needed_for_B_for_BCR_2_high, cost_B)
overrun_needed_for_B_for_BCR_1p5_high = overrun_needed_for_BCR(benefit_B_high_l_corrected, cost_needed_for_B_for_BCR_1p5_high, cost_B)

vfm_data_2 = { "A_low": [cost_A, cost_A_l_corrected, benefit_A_low, benefit_A_low_l_corrected, BCR_A_low_l_corrected, cost_needed_for_BCR_2p5_low, cost_needed_for_BCR_2_low, overrun_needed_for_BCR_2p5_low, overrun_needed_for_BCR_2_low],
                "A_high": [cost_A, cost_A_l_corrected, benefit_A_high, benefit_A_high_l_corrected, BCR_A_high_l_corrected, cost_needed_for_BCR_3_high, cost_needed_for_BCR_2p5_high, overrun_needed_for_BCR_3_high, overrun_needed_for_BCR_2p5_high],
                "B_low": [cost_B, cost_B_l_corrected, benefit_B_low, benefit_B_low_l_corrected, BCR_B_low_l_corrected, cost_needed_for_B_for_BCR_2_low, cost_needed_for_B_for_BCR_1p5_low, overrun_needed_for_B_for_BCR_2_low, overrun_needed_for_B_for_BCR_1p5_low],
                "B_high": [cost_B, cost_B_l_corrected, benefit_B_high, benefit_B_high_l_corrected, BCR_B_high_l_corrected, cost_needed_for_B_for_BCR_2_high, cost_needed_for_B_for_BCR_1p5_high, overrun_needed_for_B_for_BCR_2_high, overrun_needed_for_B_for_BCR_1p5_high]
    }

df = pd.DataFrame(vfm_data_2, index=['PVC', 'PVC_corrected', 'PVB', 'PVB_corrected', 'BCR', 'cost_needed_for_BCR_high', 'cost_needed_for_BCR_low', 'overrun_needed_for_BCR_high', 'overrun_needed_for_BCR_low'])
print(df)
'''


'\n废弃掉的代码，因为由于成本和收益都在变化，所以这种计算方法失去实际的意义\ncost_needed_for_BCR_2p5_low = cost_needed_for_BCR(benefit_A_low_l_corrected, 2.5, cost_A_l_corrected)\ncost_needed_for_BCR_3_high = cost_needed_for_BCR(benefit_A_high_l_corrected, 3, cost_A_l_corrected)\ncost_needed_for_BCR_2_low = cost_needed_for_BCR(benefit_A_low_l_corrected, 2, cost_A_l_corrected)\ncost_needed_for_BCR_2p5_high = cost_needed_for_BCR(benefit_A_high_l_corrected, 2.5, cost_A_l_corrected)\n\ncost_needed_for_B_for_BCR_2_low = cost_needed_for_BCR(benefit_B_low_l_corrected, 2, cost_B_l_corrected)\ncost_needed_for_B_for_BCR_1p5_low = cost_needed_for_BCR(benefit_B_low_l_corrected, 1.5, cost_B_l_corrected)\ncost_needed_for_B_for_BCR_2_high = cost_needed_for_BCR(benefit_B_high_l_corrected, 2, cost_B_l_corrected)\ncost_needed_for_B_for_BCR_1p5_high = cost_needed_for_BCR(benefit_B_high_l_corrected, 1.5, cost_B_l_corrected)\n\ndef overrun_needed_for_BCR(benefit_corrected, cost_need, cost):\n    return (benefit_corrected + cost_need) / cost 

时间价值，温室气体排放价值，和利率，分别改变23% 观测BCR的改变

实现途径可以是最简单的，复制几份程序，改变初始值，算出来结果即可……

1. discount_2 = discount * 1.23
2. value of time 2 = value of time / 1.23
3. emission_value (price) = emission_value (price) *1.23

因此，这部分内容见其他程序

如果所有电动汽车都变成了汽油车，会影响三部分：

1. 排放的影响，排放带来的负收益将非常严重
2. 燃油成本降低？
3. 间接税由损失转为正收入

一步步来考虑，先考虑排放的影响

In [1193]:
# litres
petrol_consumption_A = (0.45195 / average_speed__A + 0.09605 -0.00109 * average_speed__A + 0.000007 * average_speed__A ** 2) * road_length__A
petrol_consumption_B = (0.45195 / average_speed__B + 0.09605 -0.00109 * average_speed__B + 0.000007 * average_speed__B ** 2) * road_length_B
petrol_consumption_A_O = (0.45195 / average_speed__A_O + 0.09605 -0.00109 * average_speed__A_O + 0.000007 * average_speed__A_O ** 2) * road_length__O
petrol_consumption_B_O = (0.45195 / average_speed_B_O + 0.09605 -0.00109 * average_speed_B_O + 0.000007 * average_speed_B_O ** 2) * road_length__O
petrol_consumption_O = (0.45195 / average_speed__O + 0.09605 -0.00109 * average_speed__O + 0.000007 * average_speed__O ** 2) * road_length__O

petrol_kg_per_litre = 2.11

GC_petrol_emission_A = {}
GC_petrol_emission_B = {}
GC_petrol_emission_A_O = {}
GC_petrol_emission_B_O = {}
GC_petrol_emission_O = {}

for year in range(2031, 2091):
    GC_petrol_emission_A[year] = petrol_consumption_A * petrol_kg_per_litre * money_value_of_emission[year] *1.5893 / 1000
    GC_petrol_emission_B[year] = petrol_consumption_B * petrol_kg_per_litre * money_value_of_emission[year] *1.5893 / 1000
    GC_petrol_emission_A_O[year] = petrol_consumption_A_O * petrol_kg_per_litre * money_value_of_emission[year] *1.5893 / 1000
    GC_petrol_emission_B_O[year] = petrol_consumption_B_O * petrol_kg_per_litre * money_value_of_emission[year] *1.5893 / 1000
    GC_petrol_emission_O[year] = petrol_consumption_O * petrol_kg_per_litre * money_value_of_emission[year] *1.5893 / 1000

user_benefit_A_emission_low = {}
user_benefit_A_emission_high = {}
user_benefit_B_emission_low = {}
user_benefit_B_emission_high = {}

for year in range(2031, 2091):
    user_benefit_A_emission_low[year] = get_ROH(GC_petrol_emission_A[year], GC_petrol_emission_A_O[year], GC_petrol_emission_O[year], AADT__A_dict_low[year],  AADT__A_O_dict_low[year],  AADT__O_dict_low[year])
    user_benefit_A_emission_high[year] = get_ROH(GC_petrol_emission_A[year], GC_petrol_emission_A_O[year], GC_petrol_emission_O[year], AADT__A_dict_high[year],  AADT__A_O_dict_high[year],  AADT__O_dict_high[year])
    user_benefit_B_emission_low[year] = get_ROH(GC_petrol_emission_B[year], GC_petrol_emission_B_O[year], GC_petrol_emission_O[year], AADT__B_dict_low[year],  AADT__B_O_dict_low[year],  AADT__O_dict_low[year])
    user_benefit_B_emission_high[year] = get_ROH(GC_petrol_emission_B[year], GC_petrol_emission_B_O[year], GC_petrol_emission_O[year], AADT__B_dict_high[year],  AADT__B_O_dict_high[year],  AADT__O_dict_high[year])



考虑燃料成本，工作的和非工作的

In [1194]:
GC_work_petrol_fuel_price_A = {}
GC_work_petrol_fuel_price_B = {}
GC_work_petrol_fuel_price_A_O = {}
GC_work_petrol_fuel_price_B_O = {}
GC_work_petrol_fuel_price_O = {}
for year in range(2031, 2091):
    GC_work_petrol_fuel_price_A[year] = petrol_fuel_price_65[year] * road_length__A /1000
    GC_work_petrol_fuel_price_B[year] = petrol_fuel_price_65[year] * road_length_B /1000
    GC_work_petrol_fuel_price_A_O[year] = petrol_fuel_price_28[year] * road_length__O /1000
    GC_work_petrol_fuel_price_B_O[year] = petrol_fuel_price_28[year] * road_length__O /1000
    GC_work_petrol_fuel_price_O[year] = petrol_fuel_price_22[year] * road_length__O /1000

GC_non_work_petrol_fuel_price_A = {}
GC_non_work_petrol_fuel_price_B = {}
GC_non_work_petrol_fuel_price_A_O = {}
GC_non_work_petrol_fuel_price_B_O = {}
GC_non_work_petrol_fuel_price_O = {}
for year in range(2031, 2091):
    GC_non_work_petrol_fuel_price_A[year] = non_work_petrol_fuel_price_65[year] * road_length__A /1000
    GC_non_work_petrol_fuel_price_B[year] = non_work_petrol_fuel_price_65[year] * road_length_B /1000
    GC_non_work_petrol_fuel_price_A_O[year] = non_work_petrol_fuel_price_28[year] * road_length__O /1000
    GC_non_work_petrol_fuel_price_B_O[year] = non_work_petrol_fuel_price_28[year] * road_length__O /1000
    GC_non_work_petrol_fuel_price_O[year] = non_work_petrol_fuel_price_22[year] * road_length__O /1000


考虑非燃料成本，非燃料成本居然也和车的类型有关……

In [1195]:
C_65 = 4.966 + 135.946 / 65
C_28 = 4.966 + 135.946 / 28
C_22 = 4.966 + 135.946 / 22
GC_work_non_fuel_price_A = {}
GC_work_non_fuel_price_B = {}
GC_work_non_fuel_price_A_O = {}
GC_work_non_fuel_price_B_O = {}
GC_work_non_fuel_price_O = {}

for year in range(2031, 2091):
    GC_work_non_fuel_price_A[year] = C_65 * road_length__A /1000
    GC_work_non_fuel_price_B[year] = C_65 * road_length_B /1000
    GC_work_non_fuel_price_A_O[year] = C_28 * road_length__O /1000
    GC_work_non_fuel_price_B_O[year] = C_28 * road_length__O /1000
    GC_work_non_fuel_price_O[year] = C_22 * road_length__O /1000

C_2 = 3.846
GC_non_work_non_fuel_price_A = {}
GC_non_work_non_fuel_price_B = {}
GC_non_work_non_fuel_price_A_O = {}
GC_non_work_non_fuel_price_B_O = {}
GC_non_work_non_fuel_price_O = {}

for year in range(2031, 2091):
    GC_non_work_non_fuel_price_A[year] = C_2 * road_length__A /1000
    GC_non_work_non_fuel_price_B[year] = C_2 * road_length_B /1000
    GC_non_work_non_fuel_price_A_O[year] = C_2 * road_length__O /1000
    GC_non_work_non_fuel_price_B_O[year] = C_2 * road_length__O /1000
    GC_non_work_non_fuel_price_O[year] = C_2 * road_length__O /1000




考虑间接税

In [1196]:
tf = 0.2
# 计算间接税
indirect_tax_A_low_2 = {}
indirect_tax_A_high_2 = {}
indirect_tax_B_low_2 = {}
indirect_tax_B_high_2 = {}

F_work_A_low_2 = {}
F_work_A_high_2 = {}
F_work_B_low_2 = {}
F_work_B_high_2 = {}
N_work_A_low_2 ={}
N_work_A_high_2 = {}
N_work_B_low_2 = {}
N_work_B_high_2 = {}
F_non_work_A_low_2 = {}
F_non_work_A_high_2 = {}
F_non_work_B_low_2 = {}
F_non_work_B_high_2 = {}
N_non_work_A_low_2 = {}
N_non_work_A_high_2 = {}
N_non_work_B_low_2 = {}
N_non_work_B_high_2 = {}

for year in range(2031, 2091):
    F_work_A_low_2[year] = get_ROH(GC_work_petrol_fuel_price_A[year], GC_work_petrol_fuel_price_A_O[year], GC_work_petrol_fuel_price_O[year], AADT__A_dict_low[year],  AADT__A_O_dict_low[year],  AADT__O_dict_low[year])
    F_work_A_high_2[year] = get_ROH(GC_work_petrol_fuel_price_A[year], GC_work_petrol_fuel_price_A_O[year], GC_work_petrol_fuel_price_O[year], AADT__A_dict_high[year],  AADT__A_O_dict_high[year],  AADT__O_dict_high[year])
    F_work_B_low_2[year] = get_ROH(GC_work_petrol_fuel_price_B[year], GC_work_petrol_fuel_price_B_O[year], GC_work_petrol_fuel_price_O[year], AADT__B_dict_low[year],  AADT__B_O_dict_low[year],  AADT__O_dict_low[year])
    F_work_B_high_2[year] = get_ROH(GC_work_petrol_fuel_price_B[year], GC_work_petrol_fuel_price_B_O[year], GC_work_petrol_fuel_price_O[year], AADT__B_dict_high[year],  AADT__B_O_dict_high[year],  AADT__O_dict_high[year])
    N_work_A_low_2[year] = get_ROH(GC_work_non_fuel_price_A[year], GC_work_non_fuel_price_A_O[year], GC_work_non_fuel_price_O[year], AADT__A_dict_low[year],  AADT__A_O_dict_low[year],  AADT__O_dict_low[year])
    N_work_A_high_2[year] = get_ROH(GC_work_non_fuel_price_A[year], GC_work_non_fuel_price_A_O[year], GC_work_non_fuel_price_O[year], AADT__A_dict_high[year],  AADT__A_O_dict_high[year],  AADT__O_dict_high[year])
    N_work_B_low_2[year] = get_ROH(GC_work_non_fuel_price_B[year], GC_work_non_fuel_price_B_O[year], GC_work_non_fuel_price_O[year], AADT__B_dict_low[year],  AADT__B_O_dict_low[year],  AADT__O_dict_low[year])
    N_work_B_high_2[year] = get_ROH(GC_work_non_fuel_price_B[year], GC_work_non_fuel_price_B_O[year], GC_work_non_fuel_price_O[year], AADT__B_dict_high[year],  AADT__B_O_dict_high[year],  AADT__O_dict_high[year])
    F_non_work_A_low_2[year] = get_ROH(GC_non_work_petrol_fuel_price_A[year], GC_non_work_petrol_fuel_price_A_O[year], GC_non_work_petrol_fuel_price_O[year], AADT__A_dict_low[year],  AADT__A_O_dict_low[year],  AADT__O_dict_low[year])
    F_non_work_A_high_2[year] = get_ROH(GC_non_work_petrol_fuel_price_A[year], GC_non_work_petrol_fuel_price_A_O[year], GC_non_work_petrol_fuel_price_O[year], AADT__A_dict_high[year],  AADT__A_O_dict_high[year],  AADT__O_dict_high[year])
    F_non_work_B_low_2[year] = get_ROH(GC_non_work_petrol_fuel_price_B[year], GC_non_work_petrol_fuel_price_B_O[year], GC_non_work_petrol_fuel_price_O[year], AADT__B_dict_low[year],  AADT__B_O_dict_low[year],  AADT__O_dict_low[year])
    F_non_work_B_high_2[year] = get_ROH(GC_non_work_petrol_fuel_price_B[year], GC_non_work_petrol_fuel_price_B_O[year], GC_non_work_petrol_fuel_price_O[year], AADT__B_dict_high[year],  AADT__B_O_dict_high[year],  AADT__O_dict_high[year])
    N_non_work_A_low_2[year] = get_ROH(GC_non_work_non_fuel_price_A[year], GC_non_work_non_fuel_price_A_O[year], GC_non_work_non_fuel_price_O[year], AADT__A_dict_low[year],  AADT__A_O_dict_low[year],  AADT__O_dict_low[year])
    N_non_work_A_high_2[year] = get_ROH(GC_non_work_non_fuel_price_A[year], GC_non_work_non_fuel_price_A_O[year], GC_non_work_non_fuel_price_O[year], AADT__A_dict_high[year],  AADT__A_O_dict_high[year],  AADT__O_dict_high[year])
    N_non_work_B_low_2[year] = get_ROH(GC_non_work_non_fuel_price_B[year], GC_non_work_non_fuel_price_B_O[year], GC_non_work_non_fuel_price_O[year], AADT__B_dict_low[year],  AADT__B_O_dict_low[year],  AADT__O_dict_low[year])
    N_non_work_B_high_2[year] = get_ROH(GC_non_work_non_fuel_price_B[year], GC_non_work_non_fuel_price_B_O[year], GC_non_work_non_fuel_price_O[year], AADT__B_dict_high[year],  AADT__B_O_dict_high[year],  AADT__O_dict_high[year])

for year in range(2031, 2091):
    indirect_tax_A_low_2[year] = calculate_tax(1, F_work_A_low_2[year], N_work_A_low_2[year]) + calculate_tax(0, F_non_work_A_low_2[year], N_non_work_A_low_2[year])
    indirect_tax_A_high_2[year] = calculate_tax(1, F_work_A_high_2[year], N_work_A_high_2[year]) + calculate_tax(0, F_non_work_A_high_2[year], N_non_work_A_high_2[year])
    indirect_tax_B_low_2[year] = calculate_tax(1, F_work_B_low_2[year], N_work_B_low_2[year]) + calculate_tax(0, F_non_work_B_low_2[year], N_non_work_B_low_2[year])
    indirect_tax_B_high_2[year] = calculate_tax(1, F_work_B_high_2[year], N_work_B_high_2[year]) + calculate_tax(0, F_non_work_B_high_2[year], N_non_work_B_high_2[year])

print(indirect_tax_A_low_2[2090])

-72.9583242026417
